Имеются комментарии пользователей, предоставленные сервисом "Викишоп". Необходимо обучить модель различать позитивные и негативные комментарии. качество работы модели будет проверяться метрикой F1. для решения поставленной задачи выполним следующие шаги:

1. Подготовка данных. Загрузим и подготовим предоставленные данные.
2. Обучим несколько моделей.
3. Выводы. Подведем итоги полученных результатов

**1  Подготовка**

In [1]:
import nltk
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pymystem3 import Mystem
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.shape

(159292, 3)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
display(data.head())

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Проведем обработку данных - лемматизируем текст, удалим лишние символы.

In [5]:
stemmer = WordNetLemmatizer()
lemm_text = []
for sen in range(0, len(data['text'])):
    document = re.sub(r'\W', ' ', str(data.text[sen]))     
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    lemm_text.append(document)

In [ ]:
data['lemm_text'] = lemm_text
display(data.head())

Разделим выборку. Размер тестовой выборки - 20%.

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=0)

In [ ]:
features_train = train['lemm_text']
target_train = train['toxic']
features_test = test['lemm_text']
target_test = test['toxic']
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

**2  Обучение**

In [ ]:
model_lr = Pipeline([('vect', CountVectorizer()),
 ('tfidf', TfidfTransformer()),
 ('model', LogisticRegression())])
model_lr.fit(features_train, target_train)
predictions_1 = model_lr.predict(features_train)
result_1 = f1_score(target_train, predictions_1)
print(result_1)

In [ ]:
pipe_rf = Pipeline([('tfidf', TfidfVectorizer()),
 ('model', RandomForestClassifier())])
rf_params = {
 'tfidf__max_features':[2000],
 'tfidf__ngram_range': [(1, 2)],
 'tfidf__stop_words': ['english'],
 'model__max_depth': [1, 10]
}
model_rf = GridSearchCV(pipe_rf, param_grid=rf_params, cv = 5)
model_rf.fit(features_train, target_train)

In [ ]:
predictions_2 = model_rf.predict(features_train)
result_2 = f1_score(target_train, predictions_2)
print(result_2)

In [ ]:
pipe_dt = Pipeline([('tfidf', TfidfVectorizer()),
 ('model', DecisionTreeClassifier())])
dt_params = {
 'tfidf__max_features':[2000],
 'tfidf__ngram_range': [(1, 2)],
 'tfidf__stop_words': ['english'],
 'model__max_depth': [1, 10]
}
model_dt = GridSearchCV(pipe_dt, param_grid=dt_params, cv = 5)
model_dt.fit(features_train, target_train)


In [ ]:
predictions_3 = model_dt.predict(features_train)
result_3 = f1_score(target_train, predictions_3)
print(result_3)

Среди трех обученных моделей наилучший результат по метрике F1 у модели LogisticRegression.
Проверяем результаты на тестовой выборке

In [ ]:
predictions_4 = model_lr.predict(features_test)
result_4 = f1_score(target_test, predictions_4)
print(result_4)

**3  Выводы**

**Заключение**

От интернет-магазина "Викишоп" были предоставлены данные с комментариями и правками в описаниях товаров, сделанными пользователями магазина. К комментариям были проставлены оценки - позитивные или негативные.

Текст был обработан: были исключены лишние символы, все буквы приведены к строчному написанию, удалены стоп-слова. Текст был переведен в векторное представление.

Были обучены модели LogisticRegression, DeсisionTreeClassifier, Randomforest. На обучающей выборке наилучший результат по метрике F1 получился у модели LogisticRegression - 0.77. Работа модели проверена на тестовой выборке. Результат также не меньше 0,75 - 0,75083.

Для решения поставленной заказчиком задачи рекомендована к использованию модель LogisticRegression.